In [1]:
%env CYPHER=/usr/bin/cypher-shell
%env HOST=neo4j+s://7399f139.databases.neo4j.io
%env USER=neo4j
%env PW=til3M9ZugdlqvLGYo9G7VgPYyRyzEzZAoAsY0IcdTxY

env: CYPHER=/usr/bin/cypher-shell
env: HOST=neo4j+s://7399f139.databases.neo4j.io
env: USER=neo4j
env: PW=til3M9ZugdlqvLGYo9G7VgPYyRyzEzZAoAsY0IcdTxY


In [2]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"

In [3]:
from IPython.core.magic import register_cell_magic
import shlex

@register_cell_magic('neo')
def neo(line, cell):
    !{CONNECT} {shlex.quote(cell)}

### Begin Project 7 Work

#### Query 1) Find all the node labels and return each label's count

In [4]:
%%neo
MATCH (n) RETURN distinct labels(n),count(n)

+---------------------------+
| labels(n)      | count(n) |
+---------------------------+
| ["Recipe"]     | 11634    |
| ["Author"]     | 303      |
| ["Ingredient"] | 3077     |
| ["Collection"] | 1049     |
| ["DietType"]   | 12       |
+---------------------------+

5 rows
ready to start consuming query after 53 ms, results consumed after another 61 ms


#### Query 2) Find all the relationship types and return their count

In [5]:
%%neo
MATCH ()-[r]->() RETURN distinct type(r), count(r)

+----------------------------------+
| type(r)               | count(r) |
+----------------------------------+
| "WROTE"               | 11634    |
| "CONTAINS_INGREDIENT" | 106148   |
| "COLLECTION"          | 32240    |
| "DIET_TYPE"           | 14858    |
+----------------------------------+

4 rows
ready to start consuming query after 49 ms, results consumed after another 75 ms


#### Query 3) Find recipes with the diet type Low-salt and written by Orlando Murring

In [6]:
%%neo
MATCH (a:Author)-[w:WROTE]->(r:Recipe),
(r:Recipe)-[:DIET_TYPE]->(t:DietType)
WHERE t.name = "Low-salt" and a.name = "Orlando Murrin"
RETURN a.name,r.name, r.cookingTime 
ORDER BY r.cookingTime desc 
limit 10

+--------------------------------------------------------------------------------------+
| a.name           | r.name                                            | r.cookingTime |
+--------------------------------------------------------------------------------------+
| "Orlando Murrin" | "Classic Christmas pudding"                       | 28800         |
| "Orlando Murrin" | "Seven-hour lamb"                                 | 27000         |
| "Orlando Murrin" | "Braised shoulder of lamb with jewelled stuffing" | 12600         |
| "Orlando Murrin" | "Orlando Murrin's Deluxe Christmas Cake"          | 9000          |
| "Orlando Murrin" | "Roast lamb with avgolemono sauce"                | 6300          |
| "Orlando Murrin" | "Strawberry & cinnamon torte"                     | 5400          |
| "Orlando Murrin" | "Potatoes Dauphinoise"                            | 4500          |
| "Orlando Murrin" | "Buttery roast thyme potatoes"                    | 4500          |
| "Orlando Murrin" | 

In [7]:
# %%neo
# MATCH (m:Recipe)-[:DIET_TYPE]->(t:DietType) WHERE t.name = "Low-salt" WITH collect(m) as recipes
# MATCH (a:Author)-[w:WROTE]->(r:Recipe) where r in recipes RETURN r.name, r.cookingTime ORDER BY r.cookingTime desc limit 5

#### Query 4) Find the recipes pointing to collection quick chicken or has olive oil as an ingredient

In [8]:
%%neo
MATCH (r:Recipe)-[:COLLECTION]->(c:Collection),
(r:Recipe)-[:CONTAINS_INGREDIENT]->(i:Ingredient)
WHERE c.name= "Quick chicken" OR i.name = "olive oil"
RETURN distinct r.name 
order by r.name asc 
limit 10

+-----------------------------------------------+
| r.name                                        |
+-----------------------------------------------+
| "10-minute tortellini"                        |
| "10-minute tuna-bean toasts"                  |
| "15-minute chicken pasta"                     |
| "15-minute mix & match salad"                 |
| "20-minute beef in red wine"                  |
| "20-minute pork pan-fry"                      |
| "30 minute chicken & tarragon pasta"          |
| "5 easy pizza toppings"                       |
| "5-a-day chicken with kale & pistachio pesto" |
| "5-a-day couscous"                            |
+-----------------------------------------------+

10 rows
ready to start consuming query after 137 ms, results consumed after another 142 ms


#### Query 5) Find recipes without couscous and that aren't vegetarian

In [9]:
%%neo
MATCH (r:Recipe)-[:DIET_TYPE]->(d:DietType),
(r:Recipe)-[:CONTAINS_INGREDIENT]->(i:Ingredient) 
WHERE i.name <> "couscous" and d.name = "Vegetarian" 
RETURN distinct r.name, count(r.name) as Number_of_Ingredients
ORDER BY Number_of_Ingredients desc
LIMIT 10

+-------------------------------------------------------------------------------------------------+
| r.name                                                                  | Number_of_Ingredients |
+-------------------------------------------------------------------------------------------------+
| "BBC Children in Need cupcakes"                                         | 36                    |
| "Sri Lankan braised roots stew & coconut dhal dumplings"                | 32                    |
| "Creamy risotto layer cake"                                             | 30                    |
| "Veggie spiral pie with spiced tomato sauce & chopped salad"            | 29                    |
| "Persian pilaf & roasted root loaves"                                   | 28                    |
| "For-the-freezer ratatouille"                                           | 28                    |
| "Kentucky fried seitan"                                                 | 28                    |


#### Query 6) Find recipes that aren't in the healthy diet type or that aren't in the Healthy christmas side dish collection

In [10]:
%%neo
MATCH (n:Recipe)-[c1:COLLECTION]->(c:Collection),
(n:Recipe)-[d1:DIET_TYPE]->(d:DietType)
WHERE d.name <> "Healthy" or c.name <> "Healthy Christmas side dish"
RETURN n.name as recipe, c.name as collectoin, d.name as diet,ID(c1) as coll_num
limit 10

^C


#### Query 7) Find the author, collections, and ingredients of the Boxing Day soup recipe

In [11]:
%%neo
MATCH (r:Recipe)-[c:COLLECTION]->(c1:Collection),
(r:Recipe)-[ci:CONTAINS_INGREDIENT]->(ci1:Ingredient),
(a:Author)-[w:WROTE]->(r:Recipe)
WHERE r.name = "Boxing Day soup"
RETURN distinct a.name as Author, c1.name as Collections, ci1.name as Ingredients
LIMIT 10

+-----------------------------------------------------+
| Author      | Collections       | Ingredients       |
+-----------------------------------------------------+
| "Good Food" | "Leftover potato" | "curry paste"     |
| "Good Food" | "Leftover potato" | "Brussels sprout" |
| "Good Food" | "Leftover potato" | "onion"           |
| "Good Food" | "Leftover potato" | "sunflower oil"   |
| "Good Food" | "Leftover potato" | "potato"          |
| "Good Food" | "Leftover potato" | "natural yogurt"  |
| "Good Food" | "Leftover potato" | "vegetable stock" |
| "Good Food" | "Leftover potato" | "celery"          |
| "Good Food" | "Boxing Day"      | "curry paste"     |
| "Good Food" | "Boxing Day"      | "Brussels sprout" |
+-----------------------------------------------------+

10 rows
ready to start consuming query after 128 ms, results consumed after another 8 ms


#### Query 8) Find all the authors that reviewed a vegan recipe

In [12]:
%%neo
MATCH (a:Author)-[:WROTE]->(r:Recipe),
(r:Recipe)-[:DIET_TYPE]->(d:DietType)
WHERE d.name = "Vegan"
RETURN distinct a.name as Author 
ORDER BY a.name desc
LIMIT 10

+--------------------+
| Author             |
+--------------------+
| "Zetter Townhouse" |
| "Xanthe Clay"      |
| "Vasco Goncalves"  |
| "Tom Van Rooyen"   |
| "Tom Kerridge"     |
| "Tanya Maher"      |
| "Sophie Godwin"    |
| "Simon Richards"   |
| "Silvana Franco"   |
| "Sarah Cook"       |
+--------------------+

10 rows
ready to start consuming query after 83 ms, results consumed after another 12 ms


#### Query 9) Find the authors that wrote a recipe that is in the healthy diet type and also in the Healthy christmas side dish collection

In [13]:
%%neo
MATCH (a:Author)-[:WROTE]->(r:Recipe),
(r:Recipe)-[:DIET_TYPE]->(d:DietType),
(r:Recipe)-[:COLLECTION]->(c:Collection)
WHERE d.name = "Healthy" and c.name = "Healthy Christmas side dish"
RETURN distinct a.name as Authors
LIMIT 10


+-----------------+
| Authors         |
+-----------------+
| "Cathryn Evans" |
| "Gary Rhodes"   |
| "Good Food"     |
| "Sarah Cook"    |
| "Elena Silcock" |
| "John Torode"   |
| "Jenny White"   |
| "Sophie Godwin" |
| "James Martin"  |
| "Katy Gilhooly" |
+-----------------+

10 rows
ready to start consuming query after 115 ms, results consumed after another 16 ms


#### Query 10) Find the author, diet_types and collections of the Ultimate onion tart recipe

In [14]:
%%neo
MATCH (r:Recipe)-[c:COLLECTION]->(c1:Collection),
(r:Recipe)-[d:DIET_TYPE]->(d1:DietType),
(a:Author)-[w:WROTE]->(r:Recipe)
WHERE r.name = "Ultimate onion tart"
RETURN distinct a.name as Author, d1.name as Diet_Type, c1.name as Collection

+-----------------------------------------+
| Author      | Diet_Type    | Collection |
+-----------------------------------------+
| "Good Food" | "Low-salt"   | "Ultimate" |
| "Good Food" | "Low-salt"   | "Quiche"   |
| "Good Food" | "Vegetarian" | "Ultimate" |
| "Good Food" | "Vegetarian" | "Quiche"   |
+-----------------------------------------+

4 rows
ready to start consuming query after 108 ms, results consumed after another 17 ms


#### Extra Credit Work

#### Create a new node for each existing node label with at least one property

In [27]:
%%neo
// Using "Tom Kerridge" as an example:
MATCH (a:Author {name: 'Tom Kerridge'}), (r:Recipe {name: 'New Recipe'})
MERGE (a)-[:WROTE {date: '2023-11-02'}]->(r)


0 rows
ready to start consuming query after 242 ms, results consumed after another 0 ms
Created 2 relationships, Set 2 properties


#### Create a relationship for each existing relationship type with at least one propert

In [28]:
%%neo
// Add a birthdate property to "Tom Kerridge" as an example
MATCH (a:Author {name: 'Tom Kerridge'})
SET a.birthdate = '1973-07-27'


0 rows
ready to start consuming query after 56 ms, results consumed after another 0 ms
Set 1 properties


#### Update the properties of one or more nodes

In [29]:
%%neo
// Add a birthdate property to "Tom Kerridge" as an example
MATCH (a:Author {name: 'Tom Kerridge'})
SET a.birthdate = '1973-07-27'


0 rows
ready to start consuming query after 16 ms, results consumed after another 0 ms
Set 1 properties


#### Update the properties of one or more relationships

In [30]:
%%neo
// Using "Tom Kerridge" as an example to update the WROTE relationship:
MATCH (a:Author {name: 'Tom Kerridge'})-[w:WROTE]->(r:Recipe {name: 'New Recipe'})
SET w.date = '2023-11-03'


0 rows
ready to start consuming query after 66 ms, results consumed after another 0 ms
Set 2 properties


#### Add a new node label to an existing node

In [31]:
%%neo
// Add a label CelebrityChef to "Tom Kerridge"
MATCH (a:Author {name: 'Tom Kerridge'})
SET a:CelebrityChef


0 rows
ready to start consuming query after 55 ms, results consumed after another 0 ms
Added 1 labels


#### Update the relationship type of at least one relationship

In [32]:
%%neo
// Replace WROTE relationship of "Tom Kerridge" with AUTHORED
MATCH (a:Author {name: 'Tom Kerridge'})-[w:WROTE]->(r:Recipe {name: 'New Recipe'})
CREATE (a)-[:AUTHORED {date: w.date}]->(r)
DELETE w


0 rows
ready to start consuming query after 105 ms, results consumed after another 0 ms
Created 2 relationships, Deleted 2 relationships, Set 2 properties


#### Delete at least one relationship based on some criteria

In [33]:
%%neo
// Delete all WROTE relationships for "Zetter Townhouse"
MATCH (a:Author {name: 'Zetter Townhouse'})-[w:WROTE]->(r:Recipe)
DELETE w


0 rows
ready to start consuming query after 74 ms, results consumed after another 0 ms
Deleted 1 relationships


#### Delete at least one node based on some criteria

In [34]:
%%neo
// Delete "Zetter Townhouse" author node (and its relationships for safety)
MATCH (a:Author {name: 'Zetter Townhouse'})
DETACH DELETE a


0 rows
ready to start consuming query after 37 ms, results consumed after another 0 ms
Deleted 1 nodes


#### Create a unique constraint on a node propert

In [39]:
%%neo
// Find Author nodes with the same name
MATCH (a:Author)
WITH a.name AS name, COLLECT(a) AS nodes
WHERE SIZE(nodes) > 1
RETURN name, nodes


+---------------------------------------------------------------------------+
| name       | nodes                                                        |
+---------------------------------------------------------------------------+
| "Jane Doe" | [(:Author {name: "Jane Doe"}), (:Author {name: "Jane Doe"})] |
+---------------------------------------------------------------------------+

1 row
ready to start consuming query after 52 ms, results consumed after another 15 ms


In [40]:
%%neo
// Merge duplicate Author nodes (keep one and delete others)
MATCH (a:Author)
WITH a.name AS name, COLLECT(a) AS nodes
WHERE SIZE(nodes) > 1
FOREACH (n IN TAIL(nodes) | 
  DETACH DELETE n)


0 rows
ready to start consuming query after 103 ms, results consumed after another 0 ms
Deleted 1 nodes


In [41]:
%%neo
// Create a unique constraint on the name property of Author nodes
CREATE CONSTRAINT AuthorNameUnique FOR (a:Author) REQUIRE a.name IS UNIQUE


0 rows
ready to start consuming query after 55 ms, results consumed after another 0 ms
Added 1 constraints


#### Create a unique constraint on a relationship property

In [43]:
%%neo
// Create a unique constraint on the date property of WROTE relationships
CREATE CONSTRAINT WroteDateUnique FOR ()-[w:WROTE]-() REQUIRE w.date IS UNIQUE



0 rows
ready to start consuming query after 67 ms, results consumed after another 0 ms
Added 1 constraints
